# 04 Factories

So far we have looked at ways of registering classes and retrieving instances of those classes with the `ServiceContainer`. This will get us most of the way when developing a system using dependency injection, but in some situations we may want to have finer control over when the instances of our classes are created. This is typically done by using a "Factory".  

A Factory is a callable which returns an instance of a class. There are two "locations" in the dependency injection setup where we might want to define factories: 
 - during the registration of a service
 - in a service's dependencies

We may want to specify a factory to create instances of our registered service, rather than letting the `ServiceContainer` directly call it's constructor. This gives greater control over how the service instance is created.

Additionally, we may want to give a service a factory to allow it to create it's own instance (or multiple instances) of one or more of it's dependencies.

Both of these scenarios are supported with PyJudo. Let's take a look at how...

In [1]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer

In [2]:
# Interface
class IFoo(ABC):
    @abstractmethod
    def foo(self): ...

# Implementation
class Foo(IFoo):
    def __init__(self, text: str = "default"):
        self.text = text

    def foo(self):
        print(self.text)

# Factory function
def create_foo() -> Foo:
    return Foo("I was built in a factory!")

In [3]:
# Create a container and register the factory
container = ServiceContainer()

container.register(IFoo, create_foo)

In [4]:
foo = container.get(IFoo)

foo.foo()

I was built in a factory!


It's as simple as that. Instead of registering the `Foo` class, we registered a callable that returns a `Foo` and we were able to retrieve the factory built `IFoo` from the `ServiceContainer`.

There are some caveats to this... The `ServiceContainer` relies on proper type annotations in factory registration. Since it cannot check the return type of the factory before it calls it, it has to rely on the developer responsibly implementing the correct return type.  

For example, this is entirely "legal", but broken to do:

In [5]:
# Stupid is as stupid does...
container.unregister(IFoo)

def incorrect_create_foo() -> Foo:
    return "derp"

container.register(IFoo, incorrect_create_foo)

In [6]:
from pyjudo.exceptions import ServiceTypeError

try:
    foo = container.get(IFoo)
except ServiceTypeError as e:
    print(f"Error: {e!r}")


Error: ServiceTypeError("Service 'derp' is not of type 'IFoo'")


The example above has the correct annotation for the factory (returning `Foo`), so the `ServiceContainer` doesn't complain during registration, but the actual implementation returns a string. And thus, there are no errors during registration; however, when we try to retrieve `IFoo` from the `ServiceContainer`, we get a `ServiceTypeError`.

The other use case for factories is in a services dependencies.  
In some scenarios we may want to give a service a factory toallow it to create instances of it's dependencies. This is achieved in PyJudo by annotating the dependency with `Factory`:

In [7]:
from pyjudo import Factory

# Interface
class IBar(ABC): ...

# Implementation with a Factory[IFoo] dependency
class Bar(IBar):
    def __init__(self, foo_factory: Factory[IFoo]):
        self.foo_factory = foo_factory

In [8]:
container.unregister(IFoo)

container.add_transient(IFoo, Foo)
container.add_transient(IBar, Bar)

In [9]:
bar = container.get(IBar)

foo1 = bar.foo_factory()
foo1.foo()

foo2 = bar.foo_factory(text="I was built by Bar!")
foo2.foo()

default
I was built by Bar!


In this example, the implementation of `Bar` specifies `Factory[IFoo]` in it's dependencies. So, the `ServiceContainer` injects a callable (rather than an instance) into the `Bar` constructor. 

The injected factory works in a similar manner to retrieving services from the `ServiceContainer`, allowing you to pass in additional  arguments for constructing the service instance.